# <center> Transformer - Attention Is All You Need !!!  </center>

<div class="row">
  <div class="column" style = " float: center; width: 100%; padding: 5px">
    <img src="https://www.analyticsinsight.net/wp-content/uploads/2022/05/Know-About-Transformer-Machine-learning-Model-at-a-Glance-1440x564_c.jpg" alt="Positional Embedding in Encoder" style="height:100%">
  </div>
</div>

### Download and Import Necessary Packages

In [1]:
import os
from google.colab import files, drive

import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_probability as tfp
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.optimizers import Adam


from tensorflow.keras.layers import TextVectorization, Dense, MaxPooling1D, Conv1D, LSTM, MultiHeadAttention, Flatten
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


### Connect Google Colab TPU

In [2]:
os.environ['TPU_NAME']

'grpc://10.43.6.242:8470'

In [3]:
tpu_address = os.environ['TPU_NAME']
tpu = tf.distribute.cluster_resolver.TPUClusterResolver(tpu_address)
tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)

strategy = tf.distribute.experimental.TPUStrategy(tpu)

print(f"TPU with address-  {tpu.cluster_spec().as_dict()['worker']} successfully connected")
print(f"Number Of Accelerators- {strategy.num_replicas_in_sync}")

INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Initializing the TPU system: grpc://10.43.6.242:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.43.6.242:8470


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


TPU with address-  ['10.43.6.242:8470'] successfully connected
Number Of Accelerators- 8


### Dataset

In [4]:
dir_path = '/content/drive/My Drive/ML Datasets/Movie Review/100_512'

EMBEDDING_MATRIX = np.load(dir_path + '/Embedding_Matrix_100.npy')
X_train = np.load(dir_path + '/X_train.npy')
Y_train = np.load(dir_path + '/Y_train.npy')

X_test = np.load(dir_path + '/X_test.npy')
Y_test = np.load(dir_path + '/Y_test.npy')

In [5]:
print(type(X_train))
print(X_train.shape, Y_train.shape)
print(X_test.shape, Y_test.shape)

<class 'numpy.ndarray'>
(40000, 512) (40000, 1)
(10000, 512) (10000, 1)


### Define Constants
    -> In the Training loop, we use 100-Dimensional Word Embeddings and keep max length of words to be 512.
    -> So we keep only the first 512 non stop words for the classification.
    -> Since the task is a simple Classification Task, our model should generalize with this data.


In [6]:
BATCH_SIZE = 128
MAX_SEQ_LEN = 512
EMB_DIMS = 100
VOCAB_SIZE = EMBEDDING_MATRIX.shape[0]
print("Batch Size- ", BATCH_SIZE)
print("Maximum Sequence Length- ", MAX_SEQ_LEN)
print("vocabulary size-", VOCAB_SIZE)
print("Embedding Dimensions- ", EMB_DIMS)

Batch Size-  128
Maximum Sequence Length-  512
vocabulary size- 81657
Embedding Dimensions-  100


In [7]:
def get_final_dataset(dataset):
    dataset = dataset.batch(BATCH_SIZE, drop_remainder = True)
    dataset = dataset.prefetch(tf.data.AUTOTUNE)
    dataset = dataset.cache()
    return dataset

In [8]:
train_dataset = tf.data.Dataset.from_tensor_slices((X_train[:, :512], Y_train)).shuffle(40000)
val_dataset = tf.data.Dataset.from_tensor_slices((X_test[:, :512], Y_test)).shuffle(10000)
train_dataset = get_final_dataset(train_dataset)
val_dataset = get_final_dataset(val_dataset)


### Masks

In [9]:
def create_padding_mask(seq):
    seq = tf.cast(tf.math.equal(seq, 0), tf.float32)
    ## Adding required dimensions to the padding
    return seq[:, tf.newaxis, tf.newaxis, :]        # (Batch_size, 1, 1, seq_len)


# Working Example
x = tf.constant([[7, 6, 0, 0, 1], [1, 2, 3, 0, 0], [0, 0, 0, 4, 5]])
create_padding_mask(x)

<tf.Tensor: shape=(3, 1, 1, 5), dtype=float32, numpy=
array([[[[0., 0., 1., 1., 0.]]],


       [[[0., 0., 0., 1., 1.]]],


       [[[1., 1., 1., 0., 0.]]]], dtype=float32)>

## <center style = 'font-family:"courier"'> **Transformer Encoder** </center>
<center>
    <img src = "https://www.researchgate.net/publication/334288604/figure/fig1/AS:778232232148992@1562556431066/The-Transformer-encoder-structure.ppm"> </img>
    </cener>

<p align = "justify" style = 'font-family = "courier"'>Simple RNN Networks like GRUs or LSTMs do not work very well for long sentences. Simply put forward, it is difficult to keep all the information learnt in a single vector (the activation vector, **a** of the model for long sentences. After certain timesteps, the previous information tends to be lost, while current ones being captured. To address this issue, **Attention Mechanism** was introduced which took the NLP research with a storm. Everything was about **Attention** then.
In 2017 a research paper was published, namely **Attention Is All You Need**. This was when Transformers came into existance. Transformers tried to combine the characteristics of **Convolutional Neural Networks and Attention Mechanism**. 
One main difference  between other architectures and transformer is that the **input sequence can be passed parallelly** so that GPU can be used effectively and the speed of training can also be increased. It is also based on the multi-headed attention layer, so it easily overcomes the vanishing gradient issue. Transformers made **Vectorization** possible for working on Text Data and thus increasing the speed of of training.</p>

### Positional Embeddings
<p align = "justify">In the earlier approach, for sequential models, inputs were fed in sequence, so the effect of positions were automatically captured by the learning algorithm. But, in **Transformer Architecture**, inputs were all fed at once, so making it difficult for the architecture to guess the impact of position of a word in the sentence/paragraph. Thus, **Positional Encodings** were brought in.
The Inputs after being converted into Embedded Matrices are added with the positional embedding matrix for the architecture to have knowledge about the position of the words.

    
<div class="row">
  <div class="column" style = " float: left; width: 30%; padding: 5px">
    <img src="https://vaclavkosar.com/images/transformer-positional-embeddings.png" alt="Positional Embedding in Encoder">
  </div>
  <div class="column" style = "float: left; width: 50%; padding: 5px">
    <img src="https://jinglescode.github.io/assets/img/posts/illustrated-guide-transformer-10.jpg" alt="Embeddings working">
  </div>
</div>

In [10]:
def get_positional_embedding(num_positions:int, dimensions:int):
    """
    num_positions: Length Of Sequences in the dataset after padding
    dimensions: Number of dimensions used to represent each word in embedding matrix
    """
    # Create a column vector for positions
    pos_vec = np.arange(num_positions)[:, np.newaxis]
    
    # Create a row vector for dimensions
    dims_vec = np.arange(dimensions)[np.newaxis, :]
    
    i = dims_vec // 2
    angles = pos_vec * 1.0 / (pow(10_000, 2 * i / dimensions))
    angles[:, 0::2] = np.sin(angles[:, 0::2])
    angles[:, 0::1] = np.cos(angles[:, 0::1])
    pos_encoding = angles[np.newaxis, ...]
    return tf.cast(pos_encoding, tf.float32)

In [11]:
get_positional_embedding(10, 4)

<tf.Tensor: shape=(1, 10, 4), dtype=float32, numpy=
array([[[ 1.        ,  1.        ,  1.        ,  1.        ],
        [ 0.66636676,  0.5403023 ,  0.99995   ,  0.99995   ],
        [ 0.6143003 , -0.41614684,  0.9998    ,  0.9998    ],
        [ 0.9900591 , -0.9899925 ,  0.99955016,  0.99955004],
        [ 0.7270351 , -0.6536436 ,  0.9992005 ,  0.9992001 ],
        [ 0.5744009 ,  0.2836622 ,  0.9987513 ,  0.99875027],
        [ 0.9612168 ,  0.96017027,  0.9982027 ,  0.99820054],
        [ 0.7918362 ,  0.75390226,  0.997555  ,  0.997551  ],
        [ 0.5492263 , -0.14550003,  0.9968085 ,  0.99680173],
        [ 0.9162743 , -0.91113025,  0.99596363,  0.9959527 ]]],
      dtype=float32)>

### Encoder Layer

In [12]:
class EncoderLayer(tf.keras.layers.Layer):
    def __init__(self, embedding_dims, num_heads,
                  fully_connected_dim, dropout_rate = 0.1, 
                  layernorm_eps = 1e-6):
        super(EncoderLayer, self).__init__()
        
        self.mha = tf.keras.layers.MultiHeadAttention(num_heads,
                                                      key_dim = embedding_dims,
                                                      dropout = dropout_rate)
        
        self.dense1 = tf.keras.layers.Dense(fully_connected_dim, activation='relu')
        self.dense2 = tf.keras.layers.Dense(embedding_dims)
        
        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon = layernorm_eps)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon = layernorm_eps)
        self.dropout_ffn = tf.keras.layers.Dropout(dropout_rate)
        
    def call(self, x, padding_mask):
        attn_out = self.mha(x, x, x, padding_mask)
        attn_out = self.layernorm1(x + attn_out)
        ffn_out = self.dense1(attn_out)
        ffn_out = self.dense2(attn_out)
        ffn_out = self.layernorm2(attn_out + ffn_out)
        return attn_out

### Encoder

In [13]:
class Encoder(tf.keras.layers.Layer):
    def __init__(self, input_vocab_size, embedding_dims, num_heads,
                 fully_connected_dim, max_position_encoding, pretrained_embedding_matrix = None,
                 dropout_rate = 0.1, layernorm_eps = 1e-6):
        super(Encoder, self).__init__()
        
        self.embedding_dims = embedding_dims
        
        
        self.embeddings = tf.keras.layers.Embedding(input_vocab_size, self.embedding_dims)
        if pretrained_embedding_matrix is not None:
            self.embeddings.embeddings_initializer = tf.keras.initializers.constant(pretrained_embedding_matrix)
            self.embeddings.trainable = False
        self.positional_encoding = get_positional_embedding(max_position_encoding, self.embedding_dims)
        self.encoding_layer = EncoderLayer(embedding_dims, num_heads,
                  fully_connected_dim, dropout_rate = dropout_rate, 
                  layernorm_eps = layernorm_eps)
        self.dropout = tf.keras.layers.Dropout(dropout_rate)
    
    def call(self, x, padding_mask):
        seq_len = tf.shape(x)[1]
        
        x = self.embeddings(x)
        
        ## Scaling the embeddings
        x *= tf.math.sqrt(tf.cast(self.embedding_dims, tf.float32))
        
        ## Adding the Positional Encoding to embeddings
        x += self.positional_encoding[:, :seq_len, :]
        x = self.dropout(x)
        
        x = self.encoding_layer(x, padding_mask)
        
        return x

In [14]:
sample_encoder = Encoder(VOCAB_SIZE, EMB_DIMS, 2,
                 512, 512, pretrained_embedding_matrix = EMBEDDING_MATRIX)
temp_input = tf.random.uniform((128, 512), dtype=tf.int64, minval=0, maxval=200)
padding_mask = create_padding_mask(temp_input)
sample_encoder_output = sample_encoder(temp_input, padding_mask=padding_mask)

print(sample_encoder_output.shape)  # (batch_size, input_seq_len, d_model)

(128, 512, 100)


### Machine Learning Model

In [16]:
class Transformer_Encoder_Model(tf.keras.Model):
    def __init__(self, input_shape, input_vocab_size, embedding_dims, num_heads, fully_connected_dim, embedding_matrix = None):
        super(Transformer_Encoder_Model, self).__init__()
        self.encoder = Encoder(input_vocab_size = input_vocab_size,
                      embedding_dims = embedding_dims,
                      num_heads = num_heads,
                     fully_connected_dim = fully_connected_dim,
                      max_position_encoding = input_shape, pretrained_embedding_matrix = embedding_matrix,
                     dropout_rate = 0.1, layernorm_eps = 1e-6
        )
        self.batchnorm1 = tf.keras.layers.BatchNormalization(axis = 1)
        self.conv1 = Conv1D(64, kernel_size = 8, strides = 4)
        self.maxpool1 = MaxPooling1D()
        self.conv2 = Conv1D(128, kernel_size = 8, strides = 4)
        self.maxpool2 = MaxPooling1D()
        self.flatten = Flatten()
        self.dense1 = Dense(512, activation = 'relu')
        self.batchnorm2 = tf.keras.layers.BatchNormalization()
        self.dense2 = Dense(32, activation = 'relu')
        self.dense3 = Dense(1, activation = 'sigmoid')

    def call(self, x):
        padding_mask = create_padding_mask(x)
        encoder_out = self.encoder(x, padding_mask)
        x  = self.batchnorm1(encoder_out)
        x = self.conv1(x)
        x = self.maxpool1(x)
        x = self.conv2(x)
        x = self.maxpool2(x)
        x = self.flatten(x)
        x = self.dense1(x)
        x = self.batchnorm2(x)
        x = self.dense2(x)
        x = self.dense3(x)
        return x


In [ ]:
# def Mymodel(input_shape, input_vocab_size, embedding_dims, embedding_matrix = None):
#     input_x = tf.keras.layers.Input(shape = input_shape)
#     encoder_out = Encoder(input_vocab_size = input_vocab_size,
#                       embedding_dims = embedding_dims,
#                       num_heads = 3,
#                      fully_connected_dim = 128,
#                       max_position_encoding = input_shape, pretrained_embedding_matrix = embedding_matrix,
#                      dropout_rate = 0.1, layernorm_eps = 1e-6
#         )(input_x)

#     x = tf.keras.layers.BatchNormalization(axis = 1)(encoder_out)
#     x = Conv1D(64, kernel_size = 8, strides = 4)(x)
#     x = MaxPooling1D()(x)
#     x = Conv1D(128, kernel_size = 8, strides = 4)(x)
#     x = MaxPooling1D()(x)
#     x = Flatten()(x)
#     x = Dense(512, activation = 'relu')(x)
#     x = tf.keras.layers.BatchNormalization()(x)
#     x = Dense(32, activation = 'relu')(x)
#     x = Dense(1, activation = 'sigmoid')(x)

#     model = tf.keras.Model(inputs = input_x, outputs = x)
#     return model

### Create Training Loop

In [17]:
with strategy.scope():
    model = Transformer_Encoder_Model(MAX_SEQ_LEN, VOCAB_SIZE,
                                  EMB_DIMS, 2,
                                  512, embedding_matrix = EMBEDDING_MATRIX)

    loss_object = tf.keras.losses.BinaryCrossentropy(reduction = tf.keras.losses.Reduction.NONE)

    def compute_loss(labels, predictions):
        per_example_loss = loss_object(labels, predictions)
        return tf.nn.compute_average_loss(per_example_loss, global_batch_size = BATCH_SIZE * strategy.num_replicas_in_sync)

    test_loss = tf.keras.metrics.Mean(name = "test_loss")
    train_accuracy = tf.keras.metrics.BinaryAccuracy(name = "train_accuracy")
    test_accuracy = tf.keras.metrics.BinaryAccuracy(name = "test_accuracy")

    lr_scheduler = tf.keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate = 0.0005,
        decay_steps =500,
        decay_rate = 0.9
    )

    optimizer = tf.keras.optimizers.Adam(learning_rate = lr_scheduler)

    @tf.function
    def distributed_training_step(datasets_inputs):
        per_replica_losses = strategy.run(train_steps, args = (datasets_inputs, ))
        print(per_replica_losses)
        return strategy.reduce(tf.distribute.ReduceOp.SUM, per_replica_losses, axis = None)
    
    @tf.function
    def distributed_test_step(datasets_inputs):
        strategy.run(test_steps, args = (datasets_inputs, ))
    
    
    def train_steps(inputs):
        text_seq, labels = inputs

        with tf.GradientTape() as tape:
            predictions = model(text_seq)
            loss = compute_loss(labels, predictions)
        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))

        train_accuracy.update_state(labels, predictions)
        return loss

    def test_steps(inputs):
        text_seq, labels = inputs
        predictions = model(text_seq)
        loss = loss_object(labels, predictions)

        test_loss.update_state(loss)
        test_accuracy.update_state(labels, predictions)

In [19]:
NUM_EPOCHS = 20
with strategy.scope():
    for epoch in range(NUM_EPOCHS):
        epoch_start = datetime.now()
        total_loss = 0.0
        num_batches = 0

        # Training Loop
        for x in train_dataset: 
            total_loss += distributed_training_step(x)
            num_batches += 1

        train_loss = total_loss / num_batches

        # Testing Loop
        for x in val_dataset:
            distributed_test_step(x)

        epoch_end = datetime.now()
        
        template = ("Epoch {}, Loss: {:.2f}, Accuracy: {:.2f}, Test Loss: {:.2f}, Test Accuracy: {:.2f}, \t Elapsed Time: {}")

        print(template.format(
            epoch + 1,
            train_loss,
            train_accuracy.result() * 100,
            test_loss.result() / strategy.num_replicas_in_sync,
            test_accuracy.result() * 100,
            (epoch_end - epoch_start).seconds
        ))

        test_loss.reset_states()
        train_accuracy.reset_states()

        test_accuracy.reset_states()

PerReplica:{
  0: Tensor("output_0_shard_0:0", shape=(), dtype=float32),
  1: Tensor("output_0_shard_1:0", shape=(), dtype=float32),
  2: Tensor("output_0_shard_2:0", shape=(), dtype=float32),
  3: Tensor("output_0_shard_3:0", shape=(), dtype=float32),
  4: Tensor("output_0_shard_4:0", shape=(), dtype=float32),
  5: Tensor("output_0_shard_5:0", shape=(), dtype=float32),
  6: Tensor("output_0_shard_6:0", shape=(), dtype=float32),
  7: Tensor("output_0_shard_7:0", shape=(), dtype=float32)
}


PerReplica:{
  0: Tensor("output_0_shard_0:0", shape=(), dtype=float32),
  1: Tensor("output_0_shard_1:0", shape=(), dtype=float32),
  2: Tensor("output_0_shard_2:0", shape=(), dtype=float32),
  3: Tensor("output_0_shard_3:0", shape=(), dtype=float32),
  4: Tensor("output_0_shard_4:0", shape=(), dtype=float32),
  5: Tensor("output_0_shard_5:0", shape=(), dtype=float32),
  6: Tensor("output_0_shard_6:0", shape=(), dtype=float32),
  7: Tensor("output_0_shard_7:0", shape=(), dtype=float32)
}
Epoch 1, Loss: 0.70, Accuracy: 52.95, Test Loss: 0.08, Test Accuracy: 66.09, 	 Elapsed Time: 20
Epoch 2, Loss: 0.50, Accuracy: 75.68, Test Loss: 0.06, Test Accuracy: 78.07, 	 Elapsed Time: 9
Epoch 3, Loss: 0.42, Accuracy: 80.98, Test Loss: 0.06, Test Accuracy: 79.07, 	 Elapsed Time: 9
Epoch 4, Loss: 0.37, Accuracy: 83.71, Test Loss: 0.05, Test Accuracy: 80.57, 	 Elapsed Time: 9
Epoch 5, Loss: 0.31, Accuracy: 86.78, Test Loss: 0.06, Test Accuracy: 80.41, 	 Elapsed Time: 9
Epoch 6, Loss: 0.26, Accuracy:

### Observations
    -> The Model is heavily overfit to the training data,
    -> We shall try a simpler model, or apply regularizations to reduce overfitting.